In [28]:
import argparse
import time
import shutil
import os
import os.path as osp
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
import torchvision.models as models
from resnext_specialist import VA
from data_cnn60 import NTUDataLoaders, AverageMeter, make_dir, get_cases, get_num_classes
from sklearn.metrics import confusion_matrix
from collections import OrderedDict
import torch.nn.functional as F
from fgar import AE, PosMmen, JPosMmen, contractive_loss, MMDLoss, multi_class_hinge_loss, triplet_loss 

# parser = argparse.ArgumentParser(description='View adaptive')
# parser.add_argument('--ss', type=int, help="split size")
# parser.add_argument('--st', type=str, help="split type")
# parser.add_argument('--dataset', type=str, help="dataset path")
# parser.add_argument('--wdir', type=str, help="directory to save weights path")
# parser.add_argument('--le', type=str, help="language embedding model")
# parser.add_argument('--ve', type=str, help="visual embedding model")
# parser.add_argument('--phase', type=str, help="train or val")
# parser.add_argument('--gpu', type=str, help="gpu device number")
# args = parser.parse_args()

gpu = '0'
ss = 5
st = 'r'
dataset_path = 'ntu_results/shift_5_r'
# wdir = gating_our
le = 'bert'
ve = 'shift'
phase = 'train'
num_classes = 60

os.environ["CUDA_VISIBLE_DEVICES"] = gpu
seed = 5
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
device = torch.device("cuda")
print(torch.cuda.device_count())


# if not os.path.exists('/ssd_scratch/cvit/pranay.gupta/language_modelling/' + wdir):
#     os.mkdir('/ssd_scratch/cvit/pranay.gupta/language_modelling/' + wdir)
# if not os.path.exists('/ssd_scratch/cvit/pranay.gupta/language_modelling/' + wdir + '/' + le):
#     os.mkdir('/ssd_scratch/cvit/pranay.gupta/language_modelling/' + wdir + '/' + le)

# trunk_checkpoint = '/ssd_scratch/cvit/pranay.gupta/ntu_results/vacnn_val_'+str(ss)+'_'+st+'/best_val_0.pth.tar'
# trunk = nn.DataParallel(VA(60), device_ids=[0, 1, 2, 3])
# trunk = trunk.to(device)
# criterion1 = nn.CrossEntropyLoss(reduction = 'none').to(device)
# trunk.load_state_dict(torch.load(trunk_checkpoint)['trunk_state_dict'], strict=False)
# trunk_optimizer = optim.Adam(trunk.parameters(), lr=0.00001, weight_decay = 0.001)
# trunk_scheduler = ReduceLROnPlateau(trunk_optimizer, mode='max', factor=0.1, patience=2, cooldown=2, verbose=True)
# print("Loaded Visual Encoder")


criterion2 = nn.MSELoss().to(device)

if ve == 'vacnn':
    vis_emb_input_size = 2048
elif ve == 'shift':
    vis_emb_input_size = 256
else: 
    pass    
    
text_hidden_size = 100
vis_hidden_size = 512
output_size = 50

if le == 'bert_large':
    noun_emb_input_size = 1024
    verb_emb_input_size = 1024
elif le == 'w2v':
    noun_emb_input_size = 300
    verb_emb_input_size = 300
else:
    pass

ntu_loaders = NTUDataLoaders(dataset_path, 'max', 1)
train_loader = ntu_loaders.get_train_loader(1024, 8)
zsl_loader = ntu_loaders.get_val_loader(1024, 8)
val_loader = ntu_loaders.get_test_loader(1024, 8)
zsl_out_loader = ntu_loaders.get_val_out_loader(1024, 8)
val_out_loader = ntu_loaders.get_test_out_loader(1024, 8)
train_size = ntu_loaders.get_train_size()
zsl_size = ntu_loaders.get_val_size()
val_size = ntu_loaders.get_test_size()
print('Train on %d samples, validate on %d samples' % (train_size, zsl_size))



nouns_vocab = np.load('nouns_vocab.npy')
verbs_vocab = np.load('verbs_vocab.npy')
nouns = nouns_vocab[np.argmax(np.load('nouns_ohe.npy'), -1)][:num_classes]
# nouns[nouns == 'object'] = '#'
# nouns[51] = 'someone'
# nouns[55] = 'object'
# nouns = np.load('nouns.npy')
verbs = verbs_vocab[np.argmax(np.load('verbs_ohe.npy'), -1)][:num_classes]
# nouns = np.load('nouns.npy')
# verbs = np.load('verbs.npy')
# prps = np.load('prepositions.npy')
labels = np.load('labels.npy')

if phase == 'val':
    gzsl_inds = np.load('./label_splits/'+ st + 's' + str(num_classes - ss) +'.npy')
    unseen_inds = np.sort(np.load('./label_splits/' + st + 'v' + str(ss) + '_0.npy'))
    seen_inds = np.load('./label_splits/'+ st + 's' + str(num_classes - ss -ss) + '_0.npy')
else:
    gzsl_inds = np.arange(num_classes)
    unseen_inds = np.sort(np.load('./label_splits/' + st + 'u' + str(ss) + '.npy'))
    seen_inds = np.load('./label_splits/'+ st + 's' + str(num_classes - ss) + '.npy')

unseen_labels = labels[unseen_inds]
seen_labels = labels[seen_inds]

unseen_nouns = nouns[unseen_inds]
unseen_verbs = verbs[unseen_inds]
# unseen_prps = prps[unseen_inds]
seen_nouns = nouns[seen_inds]
seen_verbs = verbs[seen_inds]
# seen_prps = prps[seen_inds]
verb_corp = np.unique(verbs[gzsl_inds])
noun_corp = np.unique(nouns[gzsl_inds])
# prp_corp = np.unique(prps[gzsl_inds])

# import gensim
# model = gensim.models.KeyedVectors.load_word2vec_format('/ssd_scratch/cvit/pranay.gupta/GoogleNews-vectors-negative300.bin', binary=True)


# def get_w2v(model, words):
#     emb = np.zeros([300])
#     for word in words.split():
#         emb += model[word]
#     emb /= len(words.split())
    
#     return emb


verb_emb = torch.from_numpy(np.load(le + '_verb.npy')[:num_classes, :]).view([num_classes, verb_emb_input_size])
verb_emb = verb_emb/torch.norm(verb_emb, dim = 1).view([num_classes, 1]).repeat([1, verb_emb_input_size])
noun_emb = torch.from_numpy(np.load(le + '_noun.npy')[:num_classes, :]).view([num_classes, noun_emb_input_size])
noun_emb = noun_emb/torch.norm(noun_emb, dim = 1).view([num_classes, 1]).repeat([1, noun_emb_input_size])
# prp_w2v = torch.from_numpy(np.array([get_w2v(model, i) for i in prps])).view([60, 300])
# prp_w2v = noun_emb/torch.norm(prp_w2v, dim = 1).view([60, 1]).repeat([1, 300])

unseen_verb_emb = verb_emb[unseen_inds, :]
unseen_noun_emb = noun_emb[unseen_inds, :]
# unseen_prp_w2v = prp_w2v[unseen_inds, :]

seen_verb_emb = verb_emb[seen_inds, :]
seen_noun_emb = noun_emb[seen_inds, :]
# seen_prp_w2v = prp_w2v[seen_inds, :]
print("loaded language embeddings")


def get_text_data(target, verb_emb, noun_emb):
    return verb_emb[target].view(target.shape[0], verb_emb_input_size).float(), noun_emb[target].view(target.shape[0], verb_emb_input_size).float()


def save_checkpoint(state, filename='checkpoint.pth.tar', is_best=False):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')
        
def accuracy(class_embedding, vis_trans_out, target, inds):
    inds = torch.from_numpy(inds).to(device)
    temp_vis = vis_trans_out.unsqueeze(1).expand(vis_trans_out.shape[0], class_embedding.shape[0], vis_trans_out.shape[1])
    temp_cemb = class_embedding.unsqueeze(0).expand(vis_trans_out.shape[0], class_embedding.shape[0], vis_trans_out.shape[1])
    preds = torch.argmax(torch.sum(temp_vis*temp_cemb, axis=2), axis = 1)
    acc = torch.sum(inds[preds] == target).item()/(preds.shape[0])
    return acc, torch.sum(temp_vis*temp_cemb, axis=2)

1
Train on 36548 samples, validate on 1356 samples
loaded language embeddings


In [29]:
tars = []
for i, (_, target) in enumerate(val_loader):
    tars += list(target.numpy())

In [30]:
test_y = []
for i in tars:
    if i in unseen_inds:
        test_y.append(0)
    else:
        test_y.append(1)

In [31]:
import numpy as np
test_zs = np.load('/ssd_scratch/cvit/pranay.gupta/unseen_out/cadavae_5_r_gzsl_zs.npy')
# np.array([j.cpu().detach().numpy() for i in test_zs for j in i])
# test_noun_zs = np.array([j.cpu().detach().numpy() for i in test_noun_zs for j in i])
# test_verb_zs = np.array([j.cpu().detach().numpy() for i in test_verb_zs for j in i])
# test_seen = np.array([j.cpu().detach().numpy() for i in test_seen for j in i])
test_seen = np.load('/ssd_scratch/cvit/pranay.gupta/ntu_results/shift_5_r/gtest_out.npy')

In [32]:
# test_seen.shape
test_seen.shape

(16392, 60)

In [33]:
def temp_scale(seen_features, T):
    return np.array([np.exp(i)/np.sum(np.exp(i)) for i in (seen_features + 1e-12)/T])

In [34]:
prob_test_zs = test_zs
# np.array([np.exp(i)/np.sum(np.exp(i)) for i in test_zs])
# prob_test_noun_zs = np.array([np.exp(i)/np.sum(np.exp(i)) for i in test_noun_zs])
# prob_test_verb_zs = np.array([np.exp(i)/np.sum(np.exp(i)) for i in test_verb_zs])
prob_test_seen = temp_scale(test_seen, 4)
main_prob_test_seen = np.array([np.exp(i)/np.sum(np.exp(i)) for i in test_seen])

In [35]:
feat_test_zs = np.sort(prob_test_zs, 1)[:,::-1][:,:5]
# feat_test_noun_zs = np.sort(prob_test_noun_zs, 1)[:,::-1]
# feat_test_verb_zs = np.sort(prob_test_verb_zs, 1)[:,::-1]
feat_test_seen = np.sort(prob_test_seen, 1)[:,::-1][:,:5]

In [36]:
# mod_feat_test_zs = np.concatenate([feat_test_noun_zs, feat_test_verb_zs], 1)
gating_test_x = np.concatenate([feat_test_zs, feat_test_seen], 1)
gating_test_y = test_y
# gating_train_x = np.concatenate([np.concatenate([feat_unseen_zs[train_unseen_inds, :], feat_unseen_train[train_unseen_inds, :]], 1), np.concatenate([feat_seen_zs[train_seen_inds, :], feat_seen_train[train_seen_inds, :]], 1)], 0)
# gating_train_y = [0]*len(train_unseen_inds) + [1]*len(train_seen_inds)

In [37]:
len(gating_test_y)

16392

In [38]:
import pickle as pkl
with open('/ssd_scratch/cvit/pranay.gupta/language_modelling/cada_vae_shift_5_r/gating_model_t4_thresh0.49_seen.pkl', 'rb') as f:
    gating_model = pkl.load(f)

In [39]:
prob_gate = gating_model.predict_proba(gating_test_x)

In [40]:
pred_test = 1 - prob_gate[:, 0]>0.49

In [41]:
# pred_val = gating_model.predict(gating_val_x)
np.sum(pred_test == test_y)/len(test_y)

0.7301732552464617

In [42]:
a = prob_gate
b = np.zeros(prob_gate.shape[0])

In [44]:
p_gate_seen = prob_gate[:, 1]
prob_y_given_seen = prob_test_seen + (1/60)*np.repeat((1 - p_gate_seen)[:, np.newaxis], 60, 1)

In [45]:
p_gate_unseen = prob_gate[:, 0]
prob_y_given_unseen = prob_test_zs + (1/5)*np.repeat((1 - p_gate_unseen)[:, np.newaxis], 5, 1)

In [46]:
prob_seen = prob_y_given_seen*np.repeat(p_gate_seen[:, np.newaxis], 60, 1)
prob_unseen = prob_y_given_unseen*np.repeat(p_gate_unseen[:, np.newaxis], 5, 1)

In [47]:
final_preds = []

# soft gating
# for i in range(len(prob_seen)):
#     if np.max(prob_seen[i, :]) > np.max(prob_unseen[i, :]):
#         pred = seen_inds[np.argmax(prob_seen[i, :])]
#     else:
#         pred = unseen_inds[np.argmax(prob_unseen[i, :])]
#     final_preds.append(pred)

# hard gating
# final_preds = seen_inds[np.argmax(prob_test_seen, -1)]*pred_test + unseen_inds[np.argmax(prob_test_zs, -1)]*(1-pred_test)
seen_count = 0
tot_seen = 0
unseen_count = 0
tot_unseen = 0
gseen_count = 0
gunseen_count = 0
for i in range(len(gating_test_y)):
    if pred_test[i] == 1:
        pred = seen_inds[np.argmax(prob_test_seen[i, seen_inds])]
    else:
        pred = unseen_inds[np.argmax(prob_test_zs[i, :])]
    
    if tars[i] in seen_inds:
        tot_seen += 1
        if pred_test[i] == 1:
            gseen_count += 1
        if pred == tars[i]:
            seen_count += 1
    else:
        if pred_test[i] == 0:
            gunseen_count+=1
        tot_unseen += 1
        if pred == tars[i]:
            unseen_count += 1
    final_preds.append(pred)


In [48]:
gseen_count/tot_seen

0.7255919127427507

In [49]:
gunseen_count/tot_unseen

0.7809734513274337

In [50]:
seen_acc = seen_count/tot_seen

In [51]:
seen_acc

0.6938680500133014

In [52]:
unseen_acc = unseen_count/tot_unseen

In [53]:
unseen_acc

0.6179941002949852

In [54]:
h_mean = 2*seen_acc*unseen_acc/(seen_acc + unseen_acc)

In [55]:
h_mean

0.6537369207437486

In [35]:
unseen_count

2284